In [2]:
import pandas as pd
from transformers import pipeline
import nltk

In [3]:
BART_classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
#mDeBERTa_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

In [5]:
#df=pd.read_csv('india-news-headlines.csv')
df2 = pd.read_csv('datasets/US_stocks.csv')

df2.drop(['Ticker', 'Company Name', 'Comment'], inplace=True, axis=1)

df2_sector = df2.drop(['Industry Group', 'Industry', 'Sub-Industry'], axis=1)
df2_industry_group = df2.drop(['Sector', 'Industry', 'Sub-Industry'], axis=1)
df2_industry = df2.drop(['Industry Group', 'Sector', 'Sub-Industry'], axis=1)
df2_sub_industry = df2.drop(['Industry Group', 'Industry', 'Sector'], axis=1)

#df2.head(n=10)
#df2_sector.head(n=10)
#df2_industry_group.head(n=10)
#df2_industry.head(n=10)
#df2_sub_industry.head(n=10)
#df2_sector.values[0]

num_sector=len(df2_sector['Sector'].unique())
num_industry_group=len(df2_industry_group['Industry Group'].unique())
num_industry=len(df2_industry['Industry'].unique())
num_sub_industry=len(df2_sub_industry['Sub-Industry'].unique())

print(f'Sectors:{num_sector}\nIndustry Groups:{num_industry_group}\nIndustries:{num_industry}\nSub-Industries:{num_sub_industry}')
len(df2_sector['Description'].iloc[0])



Sectors:11
Industry Groups:24
Industries:68
Sub-Industries:155


995

In [6]:
#Predict Sector with 1st version BART

labels = df2_sector['Sector'].unique()
text = df2_sector['Description'].head(n=3)

hypothesis_template = 'This text is about {}.'
predicted_labels=[]
for sentence in text:

    prediction = BART_classifier(sentence, labels, hypothesis_template=hypothesis_template, multi_label=True)
    
    predicted_labels.append(prediction['labels'][0])

NameError: name 'BART_classifier' is not defined

In [7]:
#Text tests
from nlp_utils import *
import math

#df2_sector['Description'].iloc[0]
#df2_sector.iloc[0]
dt_text_things = add_text_length(df2_sector, 'Description')
#max_len = math.floor(dt_text_things['word_count'].mean())
#max_len = dt_text_things['word_count'].m
#lst=bart(df2_sector['Description'].iloc[0:3], max_len)

#a=nltk.word_tokenize(df2_sector['Description'].iloc[4])
#print(len(a))



                            min        mean      max
char_count           145.000000  829.929707  886.000
word_count            29.000000  156.594148  212.000
sentence_count         2.000000    6.275763   12.000
avg_word_length        3.985849    5.319662    6.928
avg_sentence_lenght   13.625000   26.273055   90.000


In [6]:
#Text cleanning

#max_len = math.floor(dt_text_things['word_count'].mean())
#max_len = dt_text_things['word_count'].min()
max_len = 100
lst=bart(df2_sector['Description'].head(500), max_len)


c:\Users\User\.conda\envs\UW\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [28]:
print(f"--------------------------------------NORMAL TEXT--------------------------------------\n{df2_sector['Description'].iloc[0:1].values}")
print()
print(f'--------------------------------------SUMMAR TEXT--------------------------------------\n{lst[0]}')
print()

--------------------------------------NORMAL TEXT--------------------------------------
['Corteva, Inc. is a global provider of seed and crop protection solutions focused on the agriculture industry. Its seed segment is engaged in developing and supplying germplasm and traits that produce optimum yields for farms around the world. This segment offers trait technologies that improve resistance to weather, disease, insects and herbicides used to control weeds, and trait technologies that enhance food and nutritional characteristics, and provides digital solutions that assist farmers in decision-making. Its crop protection segment serves the global agricultural input industry with products that protect against weeds, insects and other pests, and disease, and that improve overall crop health both above and below ground through nitrogen management and seed-applied technologies. This segment provides herbicides, insecticides, nitrogen stabilizers and pasture and range management herbicides. 

In [7]:
df3 = df2.head(500)
df3.insert(1,'Description Summarized',lst)
df3.to_csv('US_stocks_500_summ_face_bart_large_cnn.csv', index=False)

In [9]:
print(df3['Description Summarized'].iloc[1])
print("Alico, Inc. is an agribusiness and land management company. The Company's principal lines of business are citrus groves and conservation. The Alico Citrus segment owns and manages citrus land in DeSoto, Polk, Collier, Hendry, Charlotte, Highlands, and Hardee Counties in Florida. The Land Management and Other Operations segment is engaged in land leasing for recreational and grazing purposes.")
#print(df2['Description'].iloc[1])

Alico, Inc. is an agribusiness and land management company. The Company's principal lines of business are citrus groves and conservation. Its segments include Alico Citrus and Land Management and Other Operations. The company owns approximately 81,000 acres of land and approximately 90,00 acres of mineral rights throughout Florida.
Alico, Inc. is an agribusiness and land management company. The Company's principal lines of business are citrus groves and conservation. The Alico Citrus segment owns and manages citrus land in DeSoto, Polk, Collier, Hendry, Charlotte, Highlands, and Hardee Counties in Florida. The Land Management and Other Operations segment is engaged in land leasing for recreational and grazing purposes.


In [8]:
#Predict Sector with 2nd version BART

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("AyoubChLin/Bart-MNLI-CNN_news")

model = AutoModelForSequenceClassification.from_pretrained("AyoubChLin/Bart-MNLI-CNN_news")
classifier = transformers.pipeline(
    "zero-shot-classification",
    model=model,
    tokenizer=tokenizer,
    device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
)

In [12]:
df_summarized = pd.read_csv('datasets/US_stocks_500_summ_default.csv')
labels = df_summarized['Sector'].unique()
sentences = df_summarized['Description Summarized'].head(n=500)
df=df_summarized.head(n=500)

In [ ]:
prediction = [classifier(sentence, labels, multi_label=True) for sentence in sentences]

In [ ]:
predicted_label=[prediction[i]['labels'][0] for i in range(df_summarized.shape[0])]
df_summarized['Predicted Label'] = predicted_label
df_summarized.head(n=10)

In [28]:
print(predicted_labels)
print(df2_sector.head(n=3))

['Industrials', 'Industrials', 'Industrials']
                                         Description            Sector
0  Corteva, Inc. is a global provider of seed and...         Materials
1  Alico, Inc. is an agribusiness and land manage...  Consumer Staples
2  Limoneira Company is primarily an agribusiness...  Consumer Staples
